# Settings all imports

In [1]:
import tqdm
import requests
import urllib.request

import numpy as np
import pandas as pd

from datetime import date, timedelta
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from bs4.element import Comment
# from google.colab import drive

# Create a web driver based on operating system. Please change the value of the `OPERATING_SYSTEM` constant to run this notebook on your system

In [2]:
OPERATING_SYSTEM = "mac"

if OPERATING_SYSTEM == "windows":
    executable_path = "drivers/windows/chromedriver.exe"
elif OPERATING_SYSTEM == "mac":
    executable_path = "drivers/mac/chromedriver"
elif OPERATING_SYSTEM == "linux":
    executable_path = "chromedriver"

base_url = 'https://www.jobs.bg/'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(executable_path=executable_path, options=chrome_options)


"""url = 'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging'
r = driver.get(url)
html = driver.execute_script("return document.documentElement.outerHTML")
soup = BeautifulSoup(html, 'html.parser')"""

'url = \'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging\'\nr = driver.get(url)\nhtml = driver.execute_script("return document.documentElement.outerHTML")\nsoup = BeautifulSoup(html, \'html.parser\')'

# Extracting the category names and ids so we can perform filters based on category

In [3]:
def extract_categories_dataframe():
    """"""
    url = "https://www.jobs.bg/index.php"
    r = driver.get(url)
    categories_button = driver.find_element_by_id("categoriesChip")
    categories_button.click()
    html = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(html, 'html.parser')

    categories_elem = soup.find('div',attrs={'id':'categoriesSelectSheet'})
    categories_elements = categories_elem.find_all("div", attrs={"class": "mdc-chip mdc-ripple-upgraded"})

    categories_info = []
    categories_info.append((56))

    for category_element in categories_elements:
        category_id = category_element['id'].split("_")[-1]
        category_id = int(category_id)
        categories_info.append(category_id)

    #categories_info = pd.DataFrame(categories_info, columns=["category_id"])

    return categories_info

In [7]:
categories_info=extract_categories_dataframe()

In [8]:
def get_position_id(text_info_elem):
    """
    """
    class_id_elem = 'card__title mdc-typography mdc-typography--headline6 text-overflow'
    id_elem = text_info_elem.find('a',attrs={'class':class_id_elem})['href']
    position_id = id_elem.lstrip('job/')
    
    return position_id

def extract_job_position_id_as_array(base_url, text_info_elem, job_company_elem):
    """
    """
    position_id = get_position_id(text_info_elem)

    current_job_info = [position_id]
    
    return current_job_info

In [9]:
def extract_jobs(start_page, end_page):
    """"""
    jobs_list = []

    for category in tqdm.tqdm(categories_info):
        
        for page in tqdm.tqdm(range(start_page, end_page)):
    
            try:
                # This is the base url looping through all categories and specified pages
                url = "https://www.jobs.bg/front_job_search.php?frompage={}&add_sh=1&categories%5B0%5D={}&term=#paging".format(page*15, category)
              
                r = driver.get(url)
                html = driver.execute_script("return document.documentElement.outerHTML")
                soup = BeautifulSoup(html, 'html.parser')
                job_infos = soup.find_all("td", attrs = {"class": "offerslistRow"})
            except Exception as e:
                print(f"Page number: {page}, url: {url}")
                print(f"Unexpected error: {e.message}")
    
            for i in range(0, 45, 3):
                try:
                    text_info_elem = job_infos[i]
                    job_logo_elem = job_infos[i+1]
                    job_company_elem = job_infos[i+2]
    
                    current_job_info = extract_job_position_id_as_array(base_url, text_info_elem, job_company_elem)
                    current_job_info = (current_job_info, category)
                    jobs_list.append(current_job_info)
                except Exception as e:
                    print(f"Curent page: {page}, problem with the {i}, {i+1}, {i+2} tds")
                    print(f"Unexpected error: {e}")
    
        df_columns = ["position_id", "category"]
    
        jobs_df = pd.DataFrame(jobs_list, columns=df_columns)
    
        return jobs_df

In [10]:
jobs = extract_jobs(0,20)

  0%|          | 0/34 [00:16<?, ?it/s]


In [11]:
jobs

,position_id,category
0,[5713292],56
1,[5695717],56
2,[5716422],56
3,[5716418],56
4,[5702659],56
...,...,...
295,[5706397],56
296,[5706293],56
297,[5706173],56
298,[5706190],56
